In [22]:
import numpy as np
# Function to calculate the distance between two cities
# Define the distance matrix with placeholder values
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30, 35, 40, 45, 50],
    [10, 0, 35, 25, 30, 20, 40, 50, 45, 35],
    [15, 35, 0, 30, 40, 45, 20, 10, 25, 30],
    [20, 25, 30, 0, 35, 15, 50, 40, 10, 20],
    [25, 30, 40, 35, 0, 25, 30, 15, 20, 10],
    [30, 20, 45, 15, 25, 0, 35, 20, 25, 30],
    [35, 40, 20, 50, 30, 35, 0, 30, 40, 45],
    [40, 50, 10, 40, 15, 20, 30, 0, 35, 25],
    [45, 45, 25, 10, 20, 25, 40, 35, 0, 15],
    [50, 35, 30, 20, 10, 30, 45, 25, 15, 0]
])


def distance(city1, city2):
    return distance_matrix[city1][city2]

# Function to calculate the total distance of a tour
def tour_distance(tour):
    total_distance = 0
    for i in range(len(tour)-1):
        total_distance += distance(tour[i], tour[i+1])
    total_distance += distance(tour[-1], tour[0])
    return total_distance

# Ant Colony Optimization function
def ant_colony_optimization(num_cities, num_ants, num_iterations, alpha, beta, rho, q0):
    # Initialize pheromone matrix
    pheromone_matrix = np.ones((num_cities, num_cities))

    best_distance = np.inf
    best_tour = []

    for iteration in range(num_iterations):
        # Initialize ant tours
        ant_tours = []

        for ant in range(num_ants):
            # Initialize ant tour
            tour = [0]  # Start at city 0 (arbitrary choice)

            # Generate the rest of the tour
            for _ in range(num_cities-1):
                current_city = tour[-1]
                unvisited_cities = list(set(range(num_cities)) - set(tour))
                probabilities = []

                # Calculate the probability of moving to each unvisited city
                for city in unvisited_cities:
                    pheromone = pheromone_matrix[current_city][city]
                    dist = distance(current_city, city)
                    probability = pheromone**alpha * (1.0/dist)**beta
                    probabilities.append(probability)

                # Select the next city based on the probabilities
                if np.random.uniform(0, 1) < q0:
                    # Exploitation: choose the city with the maximum probability
                    next_city = unvisited_cities[np.argmax(probabilities)]
                else:
                    # Exploration: choose the city using roulette wheel selection
                    probabilities = np.array(probabilities) / sum(probabilities)
                    next_city = np.random.choice(unvisited_cities, p=probabilities)

                tour.append(next_city)

            ant_tours.append(tour)

        # Update pheromone matrix
        delta_pheromone = np.zeros((num_cities, num_cities))

        for ant_tour in ant_tours:
            tour_dist = tour_distance(ant_tour)
            if tour_dist < best_distance:
                best_distance = tour_dist
                best_tour = ant_tour

            for i in range(len(ant_tour)-1):
                city1, city2 = ant_tour[i], ant_tour[i+1]
                delta_pheromone[city1][city2] += 1.0 / tour_dist
                delta_pheromone[city2][city1] += 1.0 / tour_dist

        pheromone_matrix = (1.0 - rho) * pheromone_matrix + delta_pheromone

    return best_tour, best_distance

# Example usage
# Example usage
num_cities = 10  # Number of cities
num_ants = 20  # Number of ants
num_iterations = 100  # Number of iterations
alpha = 1  # Alpha parameter controls the importance of pheromone
beta = 2  # Beta parameter controls the importance of distance
rho = 0.1  # Rho parameter controls the pheromone evaporation rate
q0 = 0.9  # Q0 parameter controls the exploration
# Call the ant_colony_optimization function with the desired parameters


best_tour, best_distance = ant_colony_optimization(num_cities, num_ants, num_iterations, alpha, beta, rho, q0)

# Print the best tour and its distance
print("Best Tour:", best_tour)
print("Best Distance:", best_distance)

Best Tour: [0, 1, 5, 3, 8, 9, 4, 7, 2, 6]
Best Distance: 160
